In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('combined_scrape_2.csv')

In [3]:
df.shape

(3543710, 4)

In [4]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [5]:
df.dtypes

Comment Text    object
Author          object
Subreddit       object
Post ID         object
dtype: object

In [6]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [7]:
users = list(set([x for x in sum_series.index.get_level_values(0)]))

In [8]:
len(users)

5292

In [9]:
political_dict={}
for user in users:
    #create target series
    inner_dict = {}
    try:
        inner_dict['yang'] = sum_series[user]['YangForPresidentHQ']
    except:
        inner_dict['yang'] = 0
    try:
        inner_dict['bernie'] = sum_series[user]['SandersForPresident']
    except:
        inner_dict['bernie'] = 0
    try:
        inner_dict['bernie'] += sum_series[user]['wayofthebern']
    except:
        inner_dict['bernie'] += 0
    try:
        inner_dict['bernie'] += sum_series[user]['Political_Revolution']
    except:
        inner_dict['bernie'] += 0
    try:
        inner_dict['pete'] = sum_series[user]['Pete_Buttigieg']
    except:
        inner_dict['pete'] = 0
    try:
        inner_dict['kamala'] = sum_series[user]['Kamala']
    except:
        inner_dict['kamala'] = 0
    try:
        inner_dict['biden'] = sum_series[user]['JoeBiden']
    except:
        inner_dict['biden'] = 0
    political_dict[user]=inner_dict


In [10]:
df_target = pd.DataFrame(political_dict).T

In [11]:
df_target.head(5)

,bernie,biden,kamala,pete,yang
ailish,10,0,0,0,0
nobodycares65,0,0,0,0,0
smohqe,9,0,0,0,0
Jaysyn4Reddit,38,0,0,4,0
dezmodez,0,0,0,0,5


In [12]:
## run up to here

In [13]:
def func(row):
    total = sum(row)
    if total < 5:
        return 'drop'
    if (row['bernie']/total >= .8) & (row['bernie']>=4):
        return 'bernie'
    if (row['biden']/total >= .8) & (row['biden']>=4):
        return 'biden'
    if (row['kamala']/total >= .8) & (row['kamala']>=4):
        return 'kamala'
    if (row['pete']/total >= .8) & (row['pete']>=4):
        return 'pete'
    if (row['yang']/total >= .8) & (row['yang']>=4):
        return 'yang'
    return 'drop'
    

In [14]:
df_target['target'] = df_target.apply(func,axis=1)

In [15]:
df_target['target'].value_counts()

drop      2593
bernie    1109
yang       871
pete       556
kamala      89
biden       74
Name: target, dtype: int64

In [16]:
df_target.drop(['bernie','biden','kamala','pete','yang'],axis=1,inplace=True)

In [17]:
df_target = df_target[df_target['target']!='drop']

In [18]:
df_target.head()

,target
ailish,bernie
smohqe,bernie
Jaysyn4Reddit,bernie
dezmodez,yang
hdkw836f,yang


In [19]:
y_train,y_test = train_test_split(df_target['target'],random_state=42,stratify = df_target['target'])

In [20]:
y_train.size

2024

In [21]:
y_test.size

675

In [23]:
pd.DataFrame(y_train).to_csv('y_all_train.csv')

In [24]:
pd.DataFrame(y_test).to_csv('y_all_test.csv')